# Memento Pattern

- __Type:__ Behavioral
- __Popularity: ★★★☆☆__
- __Complexity: ★★☆☆☆__

The Memento pattern captures and externalizes an object's internal state so that the object can be restored to this state later, without violating encapsulation.

### Intent:

The Memento pattern is a behavioral design pattern that lets you save and restore the previous state of an object without revealing the details of its implementation.

### Problem:

You need to save the state of an object at a particular point in time so you can restore it later, but direct access to the object's fields would violate encapsulation. Some scenarios include:

- Implementing undo/redo functionality
- Creating snapshots of object state for history tracking
- Allowing users to revert changes or go back to a previous state
- Implementing checkpoints in applications like text editors or games

Without this pattern, you might be forced to expose the object's internal implementation details, violating encapsulation principles.

### Solution:

Define three key roles in the Memento pattern:

1. **Originator** - The object whose state needs to be saved and restored. It creates mementos containing snapshots of its current state and uses them to restore previous states.

2. **Memento** - A simple data object that stores the state of the Originator. It provides no functionality beyond state storage and is typically immutable.

3. **Caretaker** - Responsible for keeping track of multiple mementos, managing their lifecycle, and providing access to them. The caretaker never examines or modifies the contents of the mementos.

This structure ensures encapsulation is maintained while still allowing state capture and restoration.

### Diagram:

```mermaid
classDiagram
    class Originator {
        -state
        +createMemento()
        +restore(memento)
    }
    class Memento {
        -state
        +getState()
    }
    class Caretaker {
        -mementos[]
        +addMemento(memento)
        +getMemento(index)
    }
    
    Originator ..> Memento : creates >
    Caretaker o--> Memento : stores >
```

In [2]:
import time
from typing import List, Any


# Memento - stores the internal state of the Originator
class EditorMemento:
    def __init__(self, content: str):
        self._content = content
        self._date = time.strftime("%Y-%m-%d %H:%M:%S")

    @property
    def content(self) -> str:
        return self._content

    @property
    def date(self) -> str:
        return self._date

    def __str__(self) -> str:
        return f"[{self._date}] {self._content[:15]}..."

In [3]:
# Originator - creates and uses mementos to save/restore state
class Editor:
    def __init__(self):
        self._content = ""

    @property
    def content(self) -> str:
        return self._content

    @content.setter
    def content(self, content: str):
        self._content = content

    def type(self, text: str):
        self._content += text
        print(f"Editor: Added '{text}'")
        print(f"Current content: '{self._content}'")

    def save(self) -> EditorMemento:
        print("Editor: Saving state...")
        return EditorMemento(self._content)

    def restore(self, memento: EditorMemento):
        self._content = memento.content
        print(f"Editor: Restored to '{self._content}'")

In [4]:
# Caretaker - keeps track of multiple mementos
class History:
    def __init__(self, editor: Editor):
        self._editor = editor
        self._mementos: List[EditorMemento] = []
        self._current = -1

    def backup(self):
        print("\nHistory: Saving editor state...")
        # If we've undone and now make a new change,
        # discard all future states
        if self._current < len(self._mementos) - 1:
            self._mementos = self._mementos[: self._current + 1]

        self._mementos.append(self._editor.save())
        self._current = len(self._mementos) - 1

    def undo(self):
        if self._current <= 0:
            print("\nHistory: Can't undo, at earliest state")
            return

        print("\nHistory: Undoing to previous state...")
        self._current -= 1
        memento = self._mementos[self._current]
        self._editor.restore(memento)

    def redo(self):
        if self._current >= len(self._mementos) - 1:
            print("\nHistory: Can't redo, at latest state")
            return

        print("\nHistory: Redoing to next state...")
        self._current += 1
        memento = self._mementos[self._current]
        self._editor.restore(memento)

    def show_history(self):
        print("\nHistory: Here are the saved states:")
        for i, memento in enumerate(self._mementos):
            marker = "» " if i == self._current else "  "
            print(f"{marker}{i}: {memento}")

In [5]:
# Client code
if __name__ == "__main__":
    editor = Editor()
    history = History(editor)

    # Initial content
    editor.type("Hello, ")
    history.backup()

    # Add more content
    editor.type("world!")
    history.backup()

    # Add even more content
    editor.type(" How are you today?")
    history.backup()

    # Show history
    history.show_history()

    # Undo
    history.undo()
    history.show_history()

    # Undo again
    history.undo()
    history.show_history()

    # Redo
    history.redo()
    history.show_history()

    # Make a new change after undo
    editor.type(" Welcome!")
    history.backup()
    history.show_history()

    # Try to redo (should fail)
    history.redo()

Editor: Added 'Hello, '
Current content: 'Hello, '

History: Saving editor state...
Editor: Saving state...
Editor: Added 'world!'
Current content: 'Hello, world!'

History: Saving editor state...
Editor: Saving state...
Editor: Added ' How are you today?'
Current content: 'Hello, world! How are you today?'

History: Saving editor state...
Editor: Saving state...

History: Here are the saved states:
  0: [2025-04-03 00:06:07] Hello, ...
  1: [2025-04-03 00:06:07] Hello, world!...
» 2: [2025-04-03 00:06:07] Hello, world! H...

History: Undoing to previous state...
Editor: Restored to 'Hello, world!'

History: Here are the saved states:
  0: [2025-04-03 00:06:07] Hello, ...
» 1: [2025-04-03 00:06:07] Hello, world!...
  2: [2025-04-03 00:06:07] Hello, world! H...

History: Undoing to previous state...
Editor: Restored to 'Hello, '

History: Here are the saved states:
» 0: [2025-04-03 00:06:07] Hello, ...
  1: [2025-04-03 00:06:07] Hello, world!...
  2: [2025-04-03 00:06:07] Hello, world! 

### Real-world analogies:

1. **Photograph**:
   - A photograph is like a memento - it captures a moment in time exactly as it was
   - The scene being photographed is the originator
   - A photo album serves as the caretaker, organizing and storing photographs

2. **Game Checkpoints**:
   - In video games, saving your progress creates a snapshot of your current state
   - The game state is the originator
   - The save file is the memento
   - The game's save system acts as the caretaker

### When to use:

- When you need to implement undo/redo functionality
- When you want to create snapshots of an object's state
- When direct access to an object's fields would break encapsulation
- When you need to restore an object to a previous state
- When implementing transactional behavior where operations can be rolled back

### Python-specific implementation notes:

- Python's `copy` and `deepcopy` functions can be useful for creating mementos when objects have complex structures
- The `dataclasses` module can simplify implementation of memento objects
- Python's dynamic nature allows for flexible memento implementations without rigid class hierarchies
- The `pickle` module can be used for more complex state serialization needs
- Python's property decorators help maintain encapsulation while allowing controlled access to state

### Related patterns:

- **Command Pattern**: Often used alongside Memento to implement undoable operations where Commands store Mementos before executing actions
- **Iterator Pattern**: Can be used to traverse through a collection of mementos
- **State Pattern**: While Memento captures the state of an object at a specific point in time, State pattern encapsulates state-specific behavior
- **Prototype Pattern**: Can be used to create deep copies for Mementos

### Benefits:

* **Maintains Encapsulation**: Preserves object encapsulation while allowing state capture
* **Simplifies Originator**: Removes state saving/restoring responsibilities from the Originator
* **History Management**: Supports undo/redo functionality and state history tracking
* **Snapshot Isolation**: Provides isolated snapshots of state without affecting current operations
* **Clean Separation**: Separates state management from business logic